In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 필요한 라이브러리

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
import json
from tqdm import tqdm

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout, Bidirectional

# GPU

In [ ]:
%tensorflow_version 2.x
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# 데이터 불러오기

In [ ]:
path = '/content/drive/MyDrive/동동성_자연어 기반 인공지능 산업분류 자동화/data/전처리/'

train = pd.read_csv(path + 'train_pre_다희.csv',  engine = 'python', index_col = 'AI_id')
test = pd.read_csv(path + 'test_pre_다희.csv',  engine = 'python', index_col = 'AI_id')

In [ ]:
train['label'] = train['digit_1'] + ' ' + train['digit_2'].astype(str) + ' ' + train['digit_3'].astype(str)

In [ ]:
train

,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text,label
AI_id,,,,,,,,
id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환,카센터 자동차 부분 정비 타이어 교환,S 95 952
id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매,상점 일반인 대상 채소 과일 판매,G 47 472
id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매,절단 사업체 도매 공업 고무 합성 고무 도매,G 46 467
id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치,영업 점 일반 소비자 열쇠 잠금장치,G 47 475
id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육,어린이집 보호자 위탁 취학전 아동 보육,Q 87 872
...,...,...,...,...,...,...,...,...
id_0999996,C,13,134,제품입고,워싱,청바지워싱,제품 워싱 청바지 워싱,C 13 134
id_0999997,F,42,424,현장에서,고객의요청에의해,실내인테리어,현장 고객 요청 실내 인테리어,F 42 424
id_0999998,G,47,474,영업점에서,일반소비자에게,여성의류 판매,영업 점 일반 소비자 여성 의류 판매,G 47 474


# 시드 고정

In [ ]:
# 랜덤 시드 고정
SEED_NUM = 1234
tf.random.set_seed(SEED_NUM)

# 모델 정의

In [ ]:
class RNNClassifier(tf.keras.Model):
    def __init__(self, **kargs):
        super(RNNClassifier, self).__init__(name=kargs['model_name'])
        #고차원을 저차원으로 축소
        self.embedding = layers.Embedding(input_dim=kargs['vocab_size'], output_dim=kargs['embedding_dimension'])
        #lstm layer를 2개 이상 쌓기 위해서는 앞의 output이 sequence 형태여야함
        self.Bilstm_1_layer = tf.keras.layers.Bidirectional(LSTM(kargs['lstm_dimension'], return_sequences=True))
        self.Bilstm_2_layer = tf.keras.layers.Bidirectional(LSTM(kargs['lstm_dimension'], return_sequences=True))
        self.Bilstm_3_layer = tf.keras.layers.Bidirectional(LSTM(int(kargs['lstm_dimension']/2)))
        self.dropout = layers.Dropout(kargs['dropout_rate'])#과적합 방지를 위한 dropout
        #피드 포워드 네트워크를 거치도록 dense layer 추가, 출력 차원의 수가 dense_dimension이 됨
        #피드 포워드란 이전 층에서 나오느 출력값이 층과 층 사이에서 적용되는 가중치의 영향을 받은 다음 다음층의 입력값을 들어가는 것 
        self.fc1 = layers.Dense(units=kargs['dense_dimension'], activation=tf.keras.activations.tanh)
        #출력차원이 225차원이므로 전 레이어의 유닛의 개수는 225보다 커야 병목현상이 발생하지 않음
        self.fc2 = layers.Dense(units=kargs['output_dimension'],activation=tf.keras.activations.softmax)
    
    def call(self, x):
        x = self.embedding(x)
        x = self.dropout(x)
        x = self.Bilstm_1_layer(x)
        x = self.Bilstm_2_layer(x)
        x = self.Bilstm_3_layer(x)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# label 예측

* feature, target 정의

In [ ]:
X_train = train['text']
y_train = train['label']
X_test = test['text']

* 토큰화 및 정수 인코딩
    -  Tokenizer 는 데이터에 출현하는 모든 단어의 개수를 세고 빈도 수로 정렬해서 
    - num_words 에 지정된 만큼만 숫자로 반환하고, 나머지는 0 으로 반환합니다      

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [ ]:
word_to_index = tokenizer.word_index

print(word_to_index)
print('단어의 개수: ', len(word_to_index))

{'대상': 1, '서비스': 2, '고객': 3, '일반': 4, '시설': 5, '접객': 6, '사업장': 7, '판매': 8, '운송': 9, '소비자': 10, '일반인': 11, '소매': 12, '매장': 13, '음식점': 14, '요청': 15, '제공': 16, '화물': 17, '도매': 18, '부동산': 19, '물자동차': 20, '승객': 21, '개인택시': 22, '중개': 23, '교육': 24, '제조': 25, '학원': 26, '영업장': 27, '사무실': 28, '관리': 29, '계약': 30, '용달': 31, '종교': 32, '커피': 33, '의류': 34, '학생': 35, '가공': 36, '소매업자': 37, '자동차': 38, '의뢰': 39, '가게': 40, '공사': 41, '위주': 42, '수리': 43, '부품': 44, '개별': 45, '산업': 46, '숙박': 47, '현장': 48, '활동': 49, '염색': 50, '운영': 51, '주점': 52, '환자': 53, '미용실': 54, '기계': 55, '맥주': 56, '용품': 57, '파마': 58, '건물': 59, '한식': 60, '사용': 61, '집': 62, '점': 63, '건설': 64, '요구': 65, '교회': 66, '음료': 67, '외래': 68, '절단': 69, '진료': 70, '여성': 71, '제작': 72, '업': 73, '음식': 74, '컷': 75, '기독교': 76, '전기': 77, '노래방': 78, '요원': 79, '편의점': 80, '아파트': 81, '의원': 82, '조립': 83, '계통': 84, '음식료품': 85, '개소': 86, '어린이': 87, '소주': 88, '트': 89, '식당': 90, '철': 91, '개발': 92, '금속': 93, '전자': 94, '화장품': 95, '각종': 96, '상가': 97, '건축': 98, '수수료': 99,

In [ ]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수 = 30262개
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value 

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1 # 카운트
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 30262
등장 빈도가 2번 이하인 희귀 단어의 수: 15133
단어 집합에서 희귀 단어의 비율: 50.006608948516295
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 0.32658770738781134


* 등장 빈도가 2번 이하인 희귀 단어는 제거하고 사용

In [ ]:
vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 15130


In [ ]:
tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

* train에서 등장빈도가 2회 이하인 희귀 단어로만 구성된 행 제거 = 빈 행

In [ ]:
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

In [ ]:
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(np.array(y_train), drop_train, axis=0)
print('X_train 크기: ', len(X_train))
print('y_train 크기: ', len(y_train))

X_train 크기:  999916
y_train 크기:  999916


/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:4454: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)


In [ ]:
print('train 문장 최대 길이 :{}'.format(max(len(l) for l in X_train)))
print('train 문장 평균 길이 :{}'.format(sum(map(len, X_train))/len(X_train)))
print('test 문장 최대 길이 :{}'.format(max(len(l) for l in X_test)))
print('test 문장 평균 길이 :{}'.format(sum(map(len, X_test))/len(X_test)))

train 문장 최대 길이 :26
train 문장 평균 길이 :5.829131647058353
test 문장 최대 길이 :23
test 문장 평균 길이 :6.0263


* pad sequence: 전체 단어 길이 맞춰 주기 -> 전체 길이 보다 짧으면 0으로 채워줌
    - maxlen: 최대 문장 길이
    - truncating: 문장의 길이가 maxlen보다 길 때 앞을 자를지
    - 0을 채워줄 문장 위치

In [ ]:
max_len = max(len(l) for l in X_train)

X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

print(X_train.shape)
print(X_test.shape)

(999916, 26)
(100000, 26)


In [ ]:
X_test

array([[   0,    0,    0, ...,    3,  130,    8],
       [   0,    0,    0, ...,   37,  249, 1279],
       [   0,    0,    0, ...,  259,  278,   51],
       ...,
       [   0,    0,    0, ...,   10, 1274,    8],
       [   0,    0,    0, ...,  564, 1004,   16],
       [   0,    0,    0, ..., 2082,   81,   29]], dtype=int32)

In [ ]:
le = LabelEncoder()

y_train = le.fit_transform(y_train)
y_train = to_categorical(y_train) 

In [ ]:
print(y_train.shape)
print(y_train)

(999916, 225)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


* 모델 생성 및 학습

In [ ]:
model_name = 'rnn_classifier_en'
BATCH_SIZE = 64
NUM_EPOCHS = 10
MAX_LEN = max_len

kargs = {'model_name': model_name,
        'vocab_size': vocab_size+1,
        'embedding_dimension': 512,
        'dropout_rate': 0.5,
        'lstm_dimension': 512,
        'dense_dimension': 512,
        'output_dimension':y_train.shape[1]}

# Straitified K_fold

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED_NUM)

models = []
k = 0
for train_idx, valid_idx in skf.split(X_train, y_train.argmax(1)):
    k += 1 #k번째 fold
    print('######### {}번째 fold ########'.format(k))
    # overfitting을 막기 위한 ealrystop 추가
    earlystop_callback = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=1)
    # min_delta: the threshold that triggers the termination (acc should at least improve 0.0001)
  

        
    #모델 생성
    model = RNNClassifier(**kargs)  
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
    with tf.device('/device:GPU:0'):
        history = model.fit(X_train[train_idx], y_train[train_idx], batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,
                        validation_data=(X_train[valid_idx], y_train[valid_idx]), callbacks=[earlystop_callback])
        
    model.save_weights('{}fold_model.h5'.format(k))
        
    models.append(model)


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


######### 1번째 fold ########
Epoch 1/10
12499/12499 [==============================] - 612s 48ms/step - loss: 0.6743 - acc: 0.8387 - val_loss: 0.4022 - val_acc: 0.9005
Epoch 2/10
12499/12499 [==============================] - 598s 48ms/step - loss: 0.4096 - acc: 0.8993 - val_loss: 0.3676 - val_acc: 0.9096
Epoch 3/10
12499/12499 [==============================] - 597s 48ms/step - loss: 0.3571 - acc: 0.9109 - val_loss: 0.3478 - val_acc: 0.9128
Epoch 4/10
12499/12499 [==============================] - 596s 48ms/step - loss: 0.3259 - acc: 0.9175 - val_loss: 0.3445 - val_acc: 0.9150
Epoch 5/10
12499/12499 [==============================] - 596s 48ms/step - loss: 0.3037 - acc: 0.9222 - val_loss: 0.3420 - val_acc: 0.9162
Epoch 6/10
12499/12499 [==============================] - 595s 48ms/step - loss: 0.2879 - acc: 0.9260 - val_loss: 0.3457 - val_acc: 0.9164
Epoch 7/10
12499/12499 [==============================] - 596s 48ms/step - loss: 0.2772 - acc: 0.9279 - val_loss: 0.3462 - val_acc: 0.9164

* 예측

In [ ]:
prediction = np.zeros((100000, 225))

for m in tqdm(models):
    with tf.device('/device:GPU:0'):
        pred = np.array(m.predict(X_test, batch_size = BATCH_SIZE))
        prediction += pred
prediction /= 5

y_test = []
for i in range(len(prediction)):
    y_test.append(np.argmax(prediction[i]))

y_test = le.inverse_transform(y_test)

test['label'] = y_test

100%|██████████| 5/5 [02:02<00:00, 24.56s/it]


In [ ]:
test[['digit_1', 'digit_2', 'digit_3']] = test['label'].str.split(' ', expand = True)

In [ ]:
test

,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text,label
AI_id,,,,,,,,
id_000001,I,56,561,치킨전문점에서,고객의주문에의해,치킨판매,치킨 전문점 고객 주문 판매,I 56 561
id_000002,G,46,466,산업공구,다른 소매업자에게,철물 수공구,산업 공구 소매업자 철물 수공구,G 46 466
id_000003,S,94,949,절에서,신도을 대상으로,불교단체운영,절 신도 대상 불교 단체 운영,S 94 949
id_000004,C,30,302,영업장에서,고객요구로,자동차튜닝,영업장 고객 요구 자동차 튜닝,C 30 302
id_000005,I,56,562,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공",실내 포장마차 접객 시설 소주 맥주 제공,I 56 562
...,...,...,...,...,...,...,...,...
id_099996,G,47,472,사업장에서,일반인대상으로,버섯농장,사업장 일반 대상 버섯 농장,G 47 472
id_099997,Q,86,862,한의원에서,외래환자위주고,치료,의원 외래 환자 위주 치료,Q 86 862
id_099998,G,47,478,일반점포에서,소비자에게,그림판매,일반 점포 소비자 그림 판매,G 47 478


In [ ]:
test.to_csv('/content/drive/MyDrive/동동성_자연어 기반 인공지능 산업분류 자동화/result/0413_Bilstm_5fold.csv')